In [1]:
!pip install ckonlpy

ERROR: Could not find a version that satisfies the requirement ckonlpy (from versions: none)
ERROR: No matching distribution found for ckonlpy


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')


# import re
# from konlpy.tag import Okt, Mecab
# from ckonlpy.tag import Twitter
# from hanspell import spell_checker
# from bs4 import BeautifulSoup
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize, sent_tokenize

# import nltk
# nltk.download('stopwords')

# # -------
# stopword = pd.read_csv('data/stopword.csv')
# stopword = stopword.iloc[:,0].tolist()

# import pickle
# with open( "data/add_user_dictionary.pickle", "rb" ) as file:
#     add_dict = pickle.load(file)
    
# with open('data/remove6.pickle','rb') as file:
#     rem6 = pickle.load(file)

# twitter = Twitter()
# # 특정 단어 추가
# twitter.add_dictionary(add_dict, 'Noun')

# m = Mecab(r'C:/mecab/mecab-ko-dic')

In [3]:
song_train = pd.read_json('data/song_meta2.json')


In [4]:
song_meta = song_train.copy() 

song_meta['issue_Y'] = song_meta['issue_Y'].astype(str)
song_meta['issue_m'] = song_meta['issue_m'].astype(str)
song_meta['issue_m'] = song_meta['issue_m'].apply(lambda x : '0'+x if len(x) == 1 else x)
song_meta['date'] = song_meta[['issue_Y','issue_m']].apply('-'.join, axis=1)

song_meta.drop(song_meta.sort_values('issue_date',ascending=False)[:2].index,axis=0,inplace=True)
song_meta.drop(['issue_date','issue_Y','issue_m'],inplace=True,axis=1)

song_meta.head()

,song_gn_dtl_gnr_basket,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,date
0,[GN0901],불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2014-05
1,"[GN1601, GN1606]","Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,2008-04
2,[GN0901],Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,2018-05
3,"[GN1102, GN1101]",Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,2015-10
4,"[GN1802, GN1801]",그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2011-08


In [5]:
song_meta = song_meta[['song_gn_gnr_basket','song_gn_dtl_gnr_basket','album_id','album_name','id','song_name','artist_name_basket','date']]
song_meta.rename(columns = {'song_gn_gnr_basket':'genre', 'song_gn_dtl_gnr_basket':'detail_genre', 'id':'song_id','artist_name_basket':'artist_name','date':'issue_date'},inplace=True)
song_meta['artist_name'] = song_meta['artist_name'].apply(lambda x : ", ".join(x))
song_meta.head()

,genre,detail_genre,album_id,album_name,song_id,song_name,artist_name,issue_date
0,[GN0900],[GN0901],2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,0,Feelings,Various Artists,2014-05
1,[GN1600],"[GN1601, GN1606]",376431,"Bach : Partitas Nos. 2, 3 & 4",1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",Murray Perahia,2008-04
2,[GN0900],[GN0901],4698747,Hit,2,Solsbury Hill (Remastered 2002),Peter Gabriel,2018-05
3,[GN1100],"[GN1102, GN1101]",2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,3,Feeling Right (Everything Is Nice) (Feat. Popc...,Matoma,2015-10
4,[GN1800],"[GN1802, GN1801]",2008470,그남자 그여자,4,그남자 그여자,Jude Law,2011-08


In [6]:
# 년-월 별 장르를 pivot table형태로
genre_map = song_meta[['issue_date','genre']]

genre_map_unnest = np.dstack(
    (
        np.repeat(genre_map.issue_date.values, list(map(len, genre_map.genre))), 
        np.concatenate(genre_map.genre.values)
    )
)

plylst_genre_map = pd.DataFrame(data = genre_map_unnest[0], columns = genre_map.columns)
plylst_genre_map['cnt'] = 1

genre_pivot = pd.pivot_table(plylst_genre_map, index = ['issue_date'], columns= ['genre'],values = 'cnt', aggfunc = 'sum')
genre_pivot['sum'] = genre_pivot.sum(axis=1)
# genre_pivot.reset_index(inplace=True)

In [7]:
genre_pivot

genre,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,GN0900,GN1000,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN9000,sum
issue_date,,,,,,,,,,,,,,,,,,,,,
0-00,157.0,19.0,29.0,7.0,4.0,17.0,271.0,24.0,316.0,418.0,...,9.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,3.0,2836.0
1900-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
1919-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1927-00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1931-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12,278.0,50.0,284.0,281.0,413.0,102.0,39.0,67.0,531.0,124.0,...,32.0,2.0,15.0,70.0,45.0,162.0,2.0,2.0,31.0,3905.0
2020-01,237.0,58.0,214.0,200.0,367.0,102.0,71.0,52.0,668.0,197.0,...,14.0,NaN,29.0,61.0,45.0,120.0,2.0,5.0,13.0,3725.0
2020-02,307.0,80.0,235.0,178.0,363.0,98.0,67.0,69.0,648.0,191.0,...,27.0,NaN,21.0,107.0,49.0,164.0,16.0,4.0,10.0,4132.0


In [14]:
# 월이 표시 안된 데이터
a = []
for i in(genre_pivot.index.tolist()):
    if i[-2:] == '00':
        a.append(i)
        
genre_pivot.loc[a,:]

genre,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,GN0900,GN1000,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN9000,sum
issue_date,,,,,,,,,,,,,,,,,,,,,
0-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1927-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1936-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1939-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1941-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
genre_pivot

genre,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,GN0900,GN1000,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN9000,sum
issue_date,,,,,,,,,,,,,,,,,,,,,
0-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1900-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
1919-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1927-00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1931-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12,278.0,50.0,284.0,281.0,413.0,102.0,39.0,67.0,531.0,124.0,...,32.0,2.0,15.0,70.0,45.0,162.0,2.0,2.0,31.0,3905.0
2020-01,237.0,58.0,214.0,200.0,367.0,102.0,71.0,52.0,668.0,197.0,...,14.0,NaN,29.0,61.0,45.0,120.0,2.0,5.0,13.0,3725.0
2020-02,307.0,80.0,235.0,178.0,363.0,98.0,67.0,69.0,648.0,191.0,...,27.0,NaN,21.0,107.0,49.0,164.0,16.0,4.0,10.0,4132.0
